In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys

module_path = os.path.abspath(os.path.join('../')) # or the path
sys.path.append(module_path)


import itertools
import pandas as pd
import numpy as np
 

In [2]:
# prepare two-moon data
from sklearn.datasets import make_circles, make_classification, make_moons
from sklearn.model_selection import train_test_split

X,y  = make_moons(n_samples=200,noise=0.1, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)


from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, Normalizer

mm = make_pipeline(MinMaxScaler(), Normalizer())
X_train = mm.fit_transform(X_train)
X_test = mm.transform(X_test)

In [3]:
# fit the model
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

clf = SGDClassifier(loss="log_loss", alpha=0.01, max_iter=200, fit_intercept=False, penalty=None)

# baseline
clf.fit(X_train,y_train)

# accuracy on test
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
acc = accuracy_score(y_test,y_pred)
print('acc is: ', acc)

              precision    recall  f1-score   support

           0       0.87      0.87      0.87        46
           1       0.82      0.82      0.82        34

    accuracy                           0.85        80
   macro avg       0.85      0.85      0.85        80
weighted avg       0.85      0.85      0.85        80

acc is:  0.85


In [4]:
# with Fourier features
dim_in = X.shape[1]
dim_feat = 100
Phi = samples = np.random.normal(0, 1, size=(dim_in,dim_feat))
print(Phi.shape)

X_train_fourier = np.matmul(X_train,Phi)
X_test_fourier = np.matmul(X_test,Phi)

clf = SGDClassifier(loss="log_loss", alpha=0.01, max_iter=200, fit_intercept=False, penalty=None)

# baseline
clf.fit(X_train_fourier,y_train)

# accuracy on test
y_pred = clf.predict(X_test_fourier)
#print(classification_report(y_test, y_pred))
acc = accuracy_score(y_test,y_pred)
print('acc is: ', acc)


(2, 100)
acc is:  0.85


In [5]:
# implement sigmoid
def sigmoid(z):
    return 1/(1 + np.exp(-z))

def quantile_tiles(x,phi, tau):
    
    dim_sample = x.shape[0]
    dim_in = phi.shape[0]
    dim_feat = phi.shape[1]
    
    
    tau = np.reshape(tau,(1,dim_feat))
    tau_tile = np.kron(np.ones((dim_sample,1)),tau)
    x = np.matmul(x,phi)
    print('tx shape',x.shape)
    print('tau tile shape',tau_tile.shape)
    x = sigmoid(x)
    z = np.sign(x-tau_tile)
    return z

# repeat with quantiles-based random planes


# with Fourier features
dim_in = X.shape[1]
dim_feat = 500
Phi = np.random.normal(0, 1, size=(dim_in,dim_feat))
Tau = np.random.uniform(size=(dim_feat,1))
Tau = 0.5*np.ones((dim_feat,1))
print(Phi.shape, Tau.shape)

X_train_qt = quantile_tiles(X_train,Phi,Tau)
X_test_qt = quantile_tiles(X_test,Phi,Tau)

clf = SGDClassifier(loss="log_loss", alpha=0.01, max_iter=500, fit_intercept=False, penalty=None)

# baseline
clf.fit(X_train_qt,y_train)

# accuracy on test
y_pred = clf.predict(X_test_qt)
#print(classification_report(y_test, y_pred))
acc = accuracy_score(y_test,y_pred)
print('acc is: ', acc)


(2, 500) (500, 1)
tx shape (120, 500)
tau tile shape (120, 500)
tx shape (80, 500)
tau tile shape (80, 500)
acc is:  0.75


In [6]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0).fit(X_train_qt,y_train)
y_pred = clf.predict(X_test_qt)
# accuracy on test
y_pred = clf.predict(X_test_qt)
print(classification_report(y_test, y_pred))
acc = accuracy_score(y_test,y_pred)
print('Logistic Regression Acc is: ', acc)

              precision    recall  f1-score   support

           0       0.87      0.74      0.80        46
           1       0.71      0.85      0.77        34

    accuracy                           0.79        80
   macro avg       0.79      0.80      0.79        80
weighted avg       0.80      0.79      0.79        80

Logistic Regression Acc is:  0.7875


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from encoders import A2DLayer  # Import A2DLayer

# Convert X_train and y_train to torch tensors
data = torch.tensor(X_train, dtype=torch.float32)
labels = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)

# Build the classifier model
class Classifier(nn.Module):
    def __init__(self, input_size, hidden_size, **kwargs):
        super(Classifier, self).__init__()
        self.encoder = A2DLayer(
            in_features=input_size,
            out_features=hidden_size,
            **kwargs  # Pass A2DLayer parameters as kwargs
        )
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 1)
        )
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.classifier(x)
        return x

# Define A2DLayer parameters
a2d_params = {
    'pdf': 'normal',
    'sign_fn': 'tanh',
    'cdf_fn': 'sigmoid',
    'quantile_tx': True,
    'trainable': False
}

# Instantiate the model with A2DLayer parameters
model = Classifier(
    input_size=data.shape[1],
    hidden_size=53,
    **a2d_params  # Pass the params here
)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the classifier
epochs = 1000
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(data)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

# Evaluate the model
with torch.no_grad():
    predictions = model(data)
    predicted_labels = (predictions > 0.5).float()
    accuracy = (predicted_labels == labels).float().mean()
    print("Accuracy:", accuracy.item())

Accuracy: 0.8583333492279053


In [8]:
print(model.encoder.linear.weight.data.shape)
print(model.encoder.quantile_offset)
print(model.encoder.linear.weight.grad)

torch.Size([53, 2])
Parameter containing:
tensor([0.2854, 0.3046, 0.7308, 0.4346, 0.4871, 0.7520, 0.4713, 0.5606, 0.2365,
        0.4399, 0.9327, 0.9329, 0.9569, 0.4511, 0.8281, 0.2691, 0.8794, 0.2606,
        0.8221, 0.8527, 0.5098, 0.7567, 0.2630, 0.5927, 0.5935, 0.1203, 0.1157,
        0.6415, 0.5036, 0.2830, 0.2230, 0.9103, 0.5743, 0.7862, 0.9858, 0.8879,
        0.8686, 0.7487, 0.9658, 0.8910, 0.1236, 0.1748, 0.4280, 0.4498, 0.0924,
        0.9692, 0.0435, 0.2254, 0.1050, 0.3653, 0.5765, 0.6773, 0.3423])
None


In [ ]:
print(model.encoder.linear.weight.grad)